In [1]:
!pip install torch torchvision torchaudio

Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com


In [1]:
import os
os.environ["CUDA_VISIBLE_DEVICES"]="-1"  # To disable GPU


In [3]:
!pip install torch

Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com


In [2]:
import torch

In [3]:
# Check if CUDA (GPU) is available
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print("Device:", device)


Device: cpu


In [4]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, Dropout
from tensorflow.keras.optimizers import Adam
from sklearn.metrics import accuracy_score

# Load the dataset
df = pd.read_excel("newenglish.xlsx", header=0)

# Trim the dataset to match the number of samples in the embeddings
df = df.head(48137)

# Assuming the embeddings are already saved in a numpy file
embeddings_list = np.load("Embeddings/MT5/ENGLISH/english_mt5_transliterated_train_embeds.npy")

# Assuming the labels are in the 'label' column of the dataframe
labels = df['labels']

# Encode the labels
label_encoder = LabelEncoder()
labels = label_encoder.fit_transform(labels)

# Split the data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(embeddings_list, labels, test_size=0.2, random_state=42)



/home/abcd/anaconda3/envs/abcd/lib/python3.8/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.1
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
2024-03-29 09:34:48.795389: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-03-29 09:34:49.998642: I tensorflow/core/util/util.cc:169] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-03-29 09:34:50.247235: E tensorflow/stream_executor/cuda/cuda_blas.cc:

In [5]:
print(X_train)
print(X_test)
print(y_train)
print(y_test)

[[[ 0.67532396 -0.2890389  -0.46259832 ... -1.5622584  -0.31826553
    0.33495554]
  [ 0.52619046  0.01287738  0.1697471  ... -0.48454094 -0.16798185
    0.17700337]
  [ 0.48017442 -0.04046869  0.26474506 ... -0.49144268 -0.05729972
    0.21771373]
  ...
  [ 0.84900016  0.11142331  0.0979915  ... -0.7444926  -0.14162822
   -0.05882687]
  [ 0.84900016  0.11142331  0.0979915  ... -0.7444926  -0.14162822
   -0.05882687]
  [ 0.84900016  0.11142331  0.0979915  ... -0.7444926  -0.14162822
   -0.05882687]]

 [[ 0.63992316  0.4665408   0.5416343  ... -0.30357853 -0.05422207
    0.9981067 ]
  [ 0.07218341  0.06345739  0.45447168 ...  0.12538059 -0.14510395
    0.78084123]
  [ 0.10998677  0.02757083  0.4721359  ...  0.4101771  -0.16194685
    0.63203794]
  ...
  [ 0.16864192  0.43834785  0.58422196 ...  0.08703576 -0.11511552
    0.73240125]
  [ 0.16864192  0.43834785  0.58422196 ...  0.08703576 -0.11511552
    0.73240125]
  [ 0.16864192  0.43834785  0.58422196 ...  0.08703576 -0.11511552
    0.

In [6]:


# Define a simple deep neural network model
model = Sequential([
    Flatten(input_shape=(X_train.shape[1:])),  # Flatten the 3D embeddings to 1D
    Dense(256, activation='relu'),
    Dropout(0.5),
    Dense(128, activation='relu'),
    Dropout(0.5),
    Dense(1, activation='sigmoid')
])

# Compile the model
model.compile(optimizer=Adam(learning_rate=0.001),
              loss='binary_crossentropy',
              metrics=['accuracy'])

# Train the model
model.fit(X_train, y_train, epochs=10, batch_size=32, validation_split=0.1)

# Evaluate the model on test data
test_loss, test_accuracy = model.evaluate(X_test, y_test)
print("Test Accuracy:", test_accuracy)

2024-03-29 09:38:36.054897: E tensorflow/stream_executor/cuda/cuda_driver.cc:265] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
2024-03-29 09:38:36.055014: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:169] retrieving CUDA diagnostic information for host: user
2024-03-29 09:38:36.055027: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:176] hostname: user
2024-03-29 09:38:36.055100: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:200] libcuda reported version is: 515.105.1
2024-03-29 09:38:36.055134: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:204] kernel reported version is: 515.105.1
2024-03-29 09:38:36.055142: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:310] kernel version seems to match DSO: 515.105.1
2024-03-29 09:38:36.055619: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in perfo

Epoch 1/10
1084/1084 [==============================] - 76s 69ms/step - loss: 3.6901 - accuracy: 0.5568 - val_loss: 0.6874 - val_accuracy: 0.5541
Epoch 2/10
1084/1084 [==============================] - 73s 68ms/step - loss: 0.6886 - accuracy: 0.5674 - val_loss: 0.6874 - val_accuracy: 0.5541
Epoch 3/10
1084/1084 [==============================] - 73s 68ms/step - loss: 0.6840 - accuracy: 0.5675 - val_loss: 0.6877 - val_accuracy: 0.5541
Epoch 4/10
1084/1084 [==============================] - 73s 68ms/step - loss: 0.6840 - accuracy: 0.5675 - val_loss: 0.6876 - val_accuracy: 0.5541
Epoch 5/10
1084/1084 [==============================] - 73s 68ms/step - loss: 0.6840 - accuracy: 0.5675 - val_loss: 0.6878 - val_accuracy: 0.5541
Epoch 6/10
1084/1084 [==============================] - 73s 68ms/step - loss: 0.6840 - accuracy: 0.5675 - val_loss: 0.6878 - val_accuracy: 0.5541
Epoch 7/10
1084/1084 [==============================] - 74s 68ms/step - loss: 0.6840 - accuracy: 0.5675 - val_loss: 0.6876 -

In [9]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

y_pred = (model.predict(X_test) > 0.5).astype("int32")  # Convert probabilities to binary predictions
accuracy = accuracy_score(y_test, y_pred,)
precision = precision_score(y_test, y_pred,average="weighted")
recall = recall_score(y_test, y_pred,average="weighted")
f1 = f1_score(y_test, y_pred,average="weighted")

print("Test Accuracy:", accuracy)
print("Precision:", precision)
print("Recall:", recall)
print("F1 Score:", f1)

301/301 [==============================] - 6s 19ms/step
Test Accuracy: 0.5670959700872455
Precision: 0.3215978392891941
Recall: 0.5670959700872455
F1 Score: 0.41043796350428957


/home/abcd/anaconda3/envs/abcd/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1327: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
